In [37]:
import sys
sys.path.append("lib/python3.7/site-packages/")

import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download('punkt')

# N_DOCUMENTS = 517402
N_DOCUMENTS = 100

In [38]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [39]:
# Load Dataset
X = []
name_dict = set()
for i in range(1, N_DOCUMENTS):
    f = open("emails/processed/" + str(i + 1) + ".txt", "r")
    clean_string = ""
    for line in f:
        if line[:4] == "From":
            line = line.split(" ")
            for element in line[1].split("@")[0].split("."):
                name_dict.add(element) 
        elif line[:2] == "To":
            line = line.split(" ")
            for element in line[1].split("@")[0].split("."):
                name_dict.add(element) 
        elif line[:7] == "Subject":
            line = line.split(" ")
            clean_string += line[1]
        elif line[:10] == "X-FileName":
            break
    for line in f:
        clean_string += line.strip("\n") + "\n"
    f.close()
    X.append(clean_string)

In [40]:
stop_words = set(stopwords.words('english')) 
stop_words.add("ect")
stop_words.add("hou")
stop_words.add("com")
stop_words.add("www")
stop_words.add("http")
stop_words.add("best")
stop_words.add("thanks")
stop_words.add("thank")
stop_words.add("dear")
stop_words.add("please")
for name in name_dict:
    stop_words.add(name)

lemmatizer = WordNetLemmatizer()

for i in range(len(X)):
    sent_text = nltk.sent_tokenize(X[i])
    tokenized_text = []
    
    # Remove the numbers
    for sentence in sent_text:
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        tokenized_text += list(nltk.word_tokenize(sentence))
        
    # Remove stopwords
    X[i] = [x.lower() for x in tokenized_text if x.lower() not in stop_words and len(x) > 2]
    
    # Pos Tag    
    X[i] = [x for x in nltk.pos_tag(X[i])]
    
    # Lemmatize
    X[i] = list(map(lambda x: lemmatizer.lemmatize(x[0], pos = get_wordnet_pos(x[1])), X[i]))
    
    f = open("clean/"+str(i+1)+".txt", "w")
    f.write(" ".join(X[i]))
    f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'clean/1.txt'

In [11]:
processed_emails = X
import numpy as np
import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array

In [12]:
dictionary = gensim.corpora.Dictionary(processed_emails)
dictionary.filter_extremes(no_above=0.5)
bag_of_words = [dictionary.doc2bow(email) for email in processed_emails]

In [13]:
from gensim import corpora, models
tfidf_object = models.TfidfModel(bag_of_words)
tfidf_vectors = tfidf_object[bag_of_words]

In [32]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaMulticore
from gensim.models import LsiModel
j_index = 0
for top in range(10, 15):
    lda_model = LdaMulticore(tfidf_vectors, num_topics=top, id2word=dictionary)
    lsa_model = LsiModel(tfidf_vectors,num_topics=top, id2word=dictionary) 
    lda_coherence = CoherenceModel(model=lda_model, texts = processed_emails, dictionary=dictionary, coherence='c_v')
    lsa_coherence = CoherenceModel(model=lsa_model, texts = processed_emails, dictionary=dictionary, coherence='c_v')

    lD_name = "modeltest/LDA/lda" + str(j_index) + ".model"
    lD_coh = "modeltest/LDA/lda" + str(j_index) + ".coherence"
    
    lS_name = "modeltest/LSA/lsa" + str(j_index) + ".model"
    lS_coh = "modeltest/LSA/lsa" + str(j_index) + ".coherence"

    # save the models to the disk
    lda_model.save(lD_name)
    lda_coherence.save(lD_coh)

    lsa_model.save(lS_name)
    lsa_coherence.save(lS_coh)

    j_index = j_index+1

In [28]:
for t in range(0, lda_model.num_topics-1):
    print(t)
#     print(lda_model.show_topic(t, 80))
#     print(' - '.join([v[0] for v in load_model.show_topic(t, 20)]))

0
1
2
3
4
5
6
7
8
9
10
11
12
